In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None) # suppress unnecessary warnings
import numpy as np
from os import listdir
import re

import sqlalchemy as sa
import cx_Oracle

from pandas.tseries.offsets import Day, MonthEnd
from dateutil.relativedelta import relativedelta
import datetime
import os

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from chart_studio.plotly import plot, iplot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

import sweetviz as sv

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'nypd_arrest.ipynb',
 'Onyx Data - DataDNA Dataset Challenge - October 2021 - NYPD Arrest Data (Year to Date).csv']

In [118]:
df_csv = pd.read_csv('Onyx Data - DataDNA Dataset Challenge - October 2021 - NYPD Arrest Data (Year to Date).csv')
df_csv.columns = df_csv.columns.str.lower() 

In [343]:
df = df_csv.copy()
df['arrest_date'] = pd.to_datetime(df['arrest_date'])
df['arrest_boro'] = df['arrest_boro'].replace({'K':'Brooklyn', 'B':'Bronx', 'M':'Manhattan', 
                                               'Q':'Queens', 'S':'Staten Island'})

df['jurisdiction'] = np.where(df['jurisdiction_code']>=3,'Non-NYPD Jurisdictions',df['jurisdiction_code'])
df['jurisdiction'] = df['jurisdiction'].replace({'0':'Patrol', '1':'Transit', '2':'Housing'})

#cleaning the ofns_desc for typos
df['ofns_desc'] = df['ofns_desc'].str.replace('&','AND')
df['ofns_desc'] = df['ofns_desc'].replace("OTHER STATE LAWS (NON PENAL LA",'OTHER STATE LAWS (NON PENAL LAW)')
df['ofns_desc'] = df['ofns_desc'].replace("KIDNAPPING",'KIDNAPPING AND RELATED OFFENSES')
df['ofns_desc'] = df['ofns_desc'].replace("ADMINISTRATIVE CODE",'ADMINISTRATIVE CODES')
df['ofns_desc'] = df['ofns_desc'].replace('LOITERING/GAMBLING (CARDS, DIC','LOITERING/GAMBLING; CARDS, DIC')
df['ofns_desc'] = df['ofns_desc'].str.extract('([\D]+)')

df = df.drop(columns=['pd_desc','pd_cd','law_cat_cd','law_code','jurisdiction_code',"new georeferenced column,,,",
                      'x_coord_cd','y_coord_cd','ky_cd']).dropna(subset=['ofns_desc'])

df['arrest_precinct'] = 'Precinct ' + df['arrest_precinct'].astype(str)

df['age_group'] = 'Age ' + df['age_group'].astype(str)

df = df.rename(columns={'arrest_date':'date','ofns_desc':'offense','arrest_precinct':'precinct','perp_sex':'gender',
                        'perp_race':'race','arrest_boro':'boro'})

df.head(2)

,arrest_key,date,offense,boro,precinct,age_group,gender,race,latitude,longitude,jurisdiction
0,234251824,2021-09-29,ASSAULT,Brooklyn,Precinct 67,Age 25-44,F,BLACK,40.648867,-73.950822,Patrol
1,232239512,2021-08-14,ASSAULT,Brooklyn,Precinct 67,Age 25-44,M,BLACK,40.661943,-73.921029,Patrol


In [5]:
def sviz(df):
    report = sv.analyze(df)
    report.show_html('report.html')
    
#sviz(df)

In [354]:
df_group_by = df.groupby(['date', 'offense', 'boro', 'precinct', 'age_group','gender', 'race', 'latitude', 'longitude', 
                          'jurisdiction']).agg({'arrest_key':'nunique'})
df_group_by = df_group_by.rename(columns={'arrest_key':'arrest_count'}).reset_index()
df_group_by.to_csv('df.csv',index=False)
df_group_by.head()

,date,offense,boro,precinct,age_group,gender,race,latitude,longitude,jurisdiction,arrest_count
0,2021-01-01,ALCOHOLIC BEVERAGE CONTROL LAW,Bronx,Precinct 47,Age 25-44,M,BLACK,40.885946,-73.859813,Patrol,3
1,2021-01-01,ALCOHOLIC BEVERAGE CONTROL LAW,Queens,Precinct 113,Age 45-64,M,BLACK,40.706179,-73.758061,Patrol,1
2,2021-01-01,ARSON,Bronx,Precinct 48,Age 18-24,M,BLACK,40.845383,-73.882098,Non-NYPD Jurisdictions,1
3,2021-01-01,ASSAULT,Bronx,Precinct 40,Age 18-24,F,BLACK,40.808374,-73.914108,Patrol,1
4,2021-01-01,ASSAULT,Bronx,Precinct 40,Age 25-44,F,BLACK,40.810398,-73.924895,Patrol,1


In [357]:
offense_boro = df.groupby(['offense', 'boro']).agg({'arrest_key':'nunique'})
offense_boro = offense_boro.rename(columns={'arrest_key':'arrest_count'}).reset_index()
offense_boro['join'] = 'link'
path = pd.DataFrame(data={'path': [0, 1],'join': ['link','link']})
offense_boro = offense_boro.merge(path,on='join',how='outer').drop(columns=['join'])
offense_boro.to_csv('offense_boro.csv',index=False)
offense_boro.head()

,offense,boro,arrest_count,path
0,ADMINISTRATIVE CODES,Bronx,7,0
1,ADMINISTRATIVE CODES,Bronx,7,1
2,ADMINISTRATIVE CODES,Brooklyn,40,0
3,ADMINISTRATIVE CODES,Brooklyn,40,1
4,ADMINISTRATIVE CODES,Manhattan,7,0


In [353]:
df.loc[df['precinct']=='Precinct 47'].groupby(df['date'].dt.to_period("M")).agg({'arrest_key':'nunique'})

,arrest_key
date,
2021-01,351
2021-02,282
2021-03,293
2021-04,230
2021-05,268
2021-06,282
2021-07,315
2021-08,295
2021-09,324


# Appendix

In [ ]:
a.shape()

In [334]:
boro_lat_lon = pd.DataFrame(data={'boro': ['Brooklyn', 'Bronx', 'Manhattan', 'Queens', 'Staten Island'],
                            'latitude': [40.650002,40.837048,40.776676,40.742054,40.579021 ],
                            'longitude': [-73.949997,-73.865433,-73.971321,-73.769417,-74.151535]})
boro_lat_lon

,boro,latitude,longitude
0,Brooklyn,40.650002,-73.949997
1,Bronx,40.837048,-73.865433
2,Manhattan,40.776676,-73.971321
3,Queens,40.742054,-73.769417
4,Staten Island,40.579021,-74.151535


In [107]:
#checking typos on ofns_desc
temp = df.groupby(['ky_cd']).agg({'ofns_desc':'nunique'}).rename(columns={'ofns_desc':'count'})
temp = temp.join(df[['ky_cd','ofns_desc']].set_index('ky_cd')).reset_index()
temp.loc[temp['count']>1].drop_duplicates()#.sort_values(by='ofns_desc').ofns_desc.unique()

,ky_cd,count,ofns_desc
31504,116.0,2,SEX CRIMES
31676,116.0,2,FELONY SEX CRIMES
40692,120.0,2,CHILD ABANDONMENT/NON SUPPORT
40695,120.0,2,ENDAN WELFARE INCOMP
44222,124.0,3,KIDNAPPING & RELATED OFFENSES
44232,124.0,3,KIDNAPPING
44289,124.0,3,KIDNAPPING AND RELATED OFFENSES
44294,125.0,2,NYS LAWS-UNCLASSIFIED FELONY
44299,125.0,2,OTHER STATE LAWS (NON PENAL LA
70069,343.0,2,OTHER OFFENSES RELATED TO THEF


In [307]:
spiral_stacked = df.groupby([df['date'].dt.strftime("%b"),df['precinct']]).agg({'arrest_key':'nunique'}).reset_index()
spiral_stacked['join'] = 'link'

path = pd.DataFrame(data={'path': [0, 1],'join': ['link','link']})
spiral_stacked = spiral_stacked.merge(path,on='join',how='outer')

spiral_stacked = spiral_stacked.rename(columns={'arrest_key':'arrest_count','date':'month_name'})

spiral_stacked.to_csv('spiral_stacked.csv',index=False)

spiral_stacked.head()

,month_name,precinct,arrest_count,join,path
0,Apr,1,123,link,0
1,Apr,1,123,link,1
2,Apr,5,169,link,0
3,Apr,5,169,link,1
4,Apr,6,82,link,0


In [336]:
map_boro = df.groupby(['boro']).agg({'arrest_key':'nunique'}).join(boro_lat_lon.set_index('boro'))
map_boro = map_boro.reset_index().rename(columns={'arrest_key':'arrest_count'})
map_boro['join'] = 'link'
map_boro['state'] = 'New York'
map_boro['country'] = 'United States'

padding = 36
density = pd.DataFrame(pd.Series(range(1,padding+1))).rename(columns={0:'point'})
density['join'] = 'link'

map_boro = map_boro.merge(density,on='join',how='outer')
map_boro[['endY-latitude','endX-longitude']] = map_boro[['latitude','longitude']]
map_boro.to_csv('map_boro.csv',index=False)
map_boro.head()

,boro,arrest_count,latitude,longitude,join,state,country,point,endY-latitude,endX-longitude
0,Bronx,25304,40.837048,-73.865433,link,New York,United States,1,40.837048,-73.865433
1,Bronx,25304,40.837048,-73.865433,link,New York,United States,2,40.837048,-73.865433
2,Bronx,25304,40.837048,-73.865433,link,New York,United States,3,40.837048,-73.865433
3,Bronx,25304,40.837048,-73.865433,link,New York,United States,4,40.837048,-73.865433
4,Bronx,25304,40.837048,-73.865433,link,New York,United States,5,40.837048,-73.865433


In [153]:
#map_age_race = df[['latitude','longitude','age_group','perp_race','arrest_key']]
map_age_race = df.groupby(['latitude','longitude','perp_race']).agg({'arrest_key':'nunique'})
map_age_race = map_age_race.reset_index().rename(columns={'arrest_key':'arrest_count'})
map_age_race['join'] = 'link'
map_age_race['state'] = 'New York'
map_age_race['country'] = 'United States'

padding = 36
density = pd.DataFrame(pd.Series(range(1,padding+1))).rename(columns={0:'point'})
density['join'] = 'link'

map_age_race = map_age_race.merge(density,on='join',how='outer')
map_age_race[['endY-latitude','endX-longitude']] = map_age_race[['latitude','longitude']]
map_age_race.to_csv('map_age_race.csv',index=False)
map_age_race.head()

,latitude,longitude,perp_race,arrest_count,join,state,country
0,40.499401,-74.242175,WHITE,1,link,New York,United States
1,40.500738,-74.242158,WHITE,2,link,New York,United States
2,40.500918,-74.241253,WHITE,1,link,New York,United States
3,40.501101,-74.240347,WHITE,4,link,New York,United States
4,40.501464,-74.238536,WHITE,2,link,New York,United States


In [243]:
def level(table):
    columns = ['gender','boro','age_group','race','offense']
    values = 'arrest_key'
    aggfunc = 'nunique'
    max_level = len(columns)
    min_level = 0
    data_frame = pd.DataFrame()
    for i in range(len(columns)):
        data_frame = data_frame.append(table.groupby(columns[min_level:i+1]).agg({'arrest_key':'nunique'}).reset_index())
    return data_frame

In [217]:
x = ['gender','boro','age_group','race','offense']
columns = ['gender','boro','age_group','race','offense']
max_level = len(columns)
max_level

5

In [329]:
x = df.groupby(['date','gender','age_group','race','boro','precinct']).agg({'arrest_key':'nunique'})
x = x.rename(columns={'arrest_key':'arrest_count'}).reset_index()
x['age_group'] = "Age " + x['age_group']
x['year'] = x['date'].dt.strftime("%Y")
x['month'] = x['date'].dt.strftime("%m") + '-' + x['date'].dt.strftime("%b")
x['week_no'] = 'Week ' + x['date'].dt.strftime("%V") 
x['date_only'] = x['date'].dt.strftime("%d") 
x.to_csv('flourish.csv',index=False)
#x.pivot_table(index='precinct',columns='month',values='arrest_count',aggfunc='sum').reset_index().to_csv('bar_race.csv',index=False)
x.head()

,date,gender,age_group,race,boro,precinct,arrest_count,year,month,week_no,date_only
0,2021-01-01,F,Age 18-24,ASIAN / PACIFIC ISLANDER,Brooklyn,Precinct 79,1,2021,01-Jan,Week 53,01
1,2021-01-01,F,Age 18-24,ASIAN / PACIFIC ISLANDER,Manhattan,Precinct 24,1,2021,01-Jan,Week 53,01
2,2021-01-01,F,Age 18-24,BLACK,Bronx,Precinct 40,1,2021,01-Jan,Week 53,01
3,2021-01-01,F,Age 18-24,BLACK,Bronx,Precinct 44,1,2021,01-Jan,Week 53,01
4,2021-01-01,F,Age 18-24,BLACK,Brooklyn,Precinct 67,1,2021,01-Jan,Week 53,01


In [231]:
max_level = len(columns)
min_level = 0
for i in range(len(columns)):
     print(columns[min_level:i+1])

['gender']
['gender', 'boro']
['gender', 'boro', 'age_group']
['gender', 'boro', 'age_group', 'race']
['gender', 'boro', 'age_group', 'race', 'offense']


In [189]:
lvl1 = df.groupby(['gender']).agg({'arrest_key':'nunique'}).reset_index()
lvl2 = df.groupby(['gender','boro']).agg({'arrest_key':'nunique'}).reset_index()
lvl3 = df.groupby(['gender','boro','age_group']).agg({'arrest_key':'nunique'}).reset_index()
lvl4 = df.groupby(['gender','boro','age_group','race']).agg({'arrest_key':'nunique'}).reset_index()
lvl5 = df.groupby(['gender','boro','age_group','race','offense']).agg({'arrest_key':'nunique'}).reset_index()

In [207]:
sunburst = lvl1.append(lvl2).append(lvl3).append(lvl4).append(lvl5)
sunburst['count_na'] = sunburst[['boro','age_group','race','offense']].isnull().sum(axis=1)
sunburst['level'] = (sunburst['count_na'].max()+1 )- sunburst['count_na']
sunburst = sunburst.drop(columns=['count_na'])
sunburst.head()

,gender,arrest_key,boro,age_group,race,offense,level
0,F,19442,NaN,NaN,NaN,NaN,1
1,M,95796,NaN,NaN,NaN,NaN,1
0,F,4467,Bronx,NaN,NaN,NaN,2
1,F,5022,Brooklyn,NaN,NaN,NaN,2
2,F,4853,Manhattan,NaN,NaN,NaN,2


In [169]:
x = df.groupby(['gender','boro','age_group','race','offense']).agg({'arrest_key':'nunique'}).reset_index()
x = x.rename(columns={'arrest_key':'arrest_count'})
x = df[['gender','boro','age_group','race','offense']].merge(x,on=['gender','boro','age_group','race','offense'],how='left')
x.head()

,gender,boro,age_group,race,offense,arrest_count
0,F,Brooklyn,25-44,BLACK,ASSAULT,406
1,M,Brooklyn,25-44,BLACK,ASSAULT,1225
2,M,Brooklyn,25-44,BLACK,ASSAULT,1225
3,F,Brooklyn,25-44,BLACK,ASSAULT,406
4,M,Brooklyn,25-44,BLACK,ASSAULT,1225


In [361]:
columns = ['gender','age_group']#,'offense' 'boro',
dataset = df#.head()
values = 'arrest_key'
aggfunc = 'nunique'
min_level = 0
sunburst = pd.DataFrame()

for i in range(len(columns)):
    sunburst = sunburst.append(dataset.groupby(columns[min_level:i+1]).agg({values:aggfunc}).reset_index())
    
to_pad = pd.DataFrame(data={'to_pad': [1, 203],'join': ['link','link']})

sunburst['count_na'] = sunburst.iloc[:,2:].isnull().sum(axis=1)
sunburst['level'] = (sunburst['count_na'].max()+1 )- sunburst['count_na']
sunburst = sunburst.drop(columns=['count_na'])
sunburst['join'] = 'link'
sunburst_final = sunburst.merge(to_pad,on='join',how='outer').rename(columns={'arrest_key':'arrest_count'})
sunburst_final.drop(columns=['join']).to_csv('sunburst.csv',index=False)
sunburst_final.tail()

,gender,arrest_count,age_group,level,join,to_pad
19,M,19123,Age 45-64,2,link,203
20,M,1449,Age 65+,2,link,1
21,M,1449,Age 65+,2,link,203
22,M,2834,Age <18,2,link,1
23,M,2834,Age <18,2,link,203
